In [0]:
#Connection configuration
spark.conf.set(
"fs.azure.account.key.aminbenstorage.blob.core.windows.net", "AQbPX3InwH459QioNrh/D0zeeCwhM0gACyGIfFC9UEJtJSJAe77zjhrtP2MXJe/DtCcvTfLVF5z/+AStd4j9Cg=="
)

#affichage de données
spark_df = spark.read.format('csv').option('header', True).load("wasbs://public-transport-data@aminbenstorage.blob.core.windows.net/raw/public-transport-data.csv")

display(spark_df)

In [0]:
#infos sur les colonnes
spark_df.printSchema()

In [0]:
unique_dates = spark_df.select("Date").distinct()
display(unique_dates)

In [0]:
from pyspark.sql.functions import year, month, dayofmonth, dayofweek, to_date

# Convertir la colonne "Date" en un format de date
spark_df = spark_df.withColumn("Date", to_date("Date", "yyyy-MM-dd"))

# Extraire l'année, le mois, le jour et le jour de la semaine
spark_df = spark_df.withColumn("Year", year("Date"))
spark_df = spark_df.withColumn("Month", month("Date"))
spark_df = spark_df.withColumn("Day", dayofmonth("Date"))
spark_df = spark_df.withColumn("DayOfWeek", dayofweek("Date"))

# Afficher les résultats
display(spark_df)

In [0]:
# Supprimer les lignes où les deux premiers caractères de la colonne "ArrivalTime" sont "24" ou supérieurs à "24"
spark_df = spark_df.filter(~(col("ArrivalTime").substr(1, 2) >= "24"))
spark_df = spark_df.filter(~(col("DepartureTime").substr(1, 2) >= "24"))

display(spark_df)

In [0]:
from pyspark.sql.functions import expr, unix_timestamp 
spark_df = spark_df.withColumn("Duration", expr(
    "from_unixtime(unix_timestamp(ArrivalTime, 'HH:mm') - unix_timestamp(DepartureTime, 'HH:mm'), 'HH:mm')"
))

display(spark_df)

In [0]:
from pyspark.sql.functions import when 

# Catégoriser les retards en fonction de la colonne "Delay"

spark_df = spark_df.withColumn("DelayCategory", 
                   when(col("Delay") <= 0, "No Delay")
                   .when((col("Delay") > 0) & (col("Delay") <= 10), "Short Delay")
                   .when((col("Delay") > 10) & (col("Delay") <= 20), "Medium Delay")
                   .otherwise("Long Delay"))
display(spark_df)

In [0]:
from pyspark.sql.functions import avg


average_passengers = spark_df.select(avg("Passengers")).first()[0]

# Identifier les heures de pointe et heures hors pointe en fonction du nombre de passagers :

spark_df = spark_df.withColumn("HeureDePointe", when(col("Passengers") > average_passengers, "peak").otherwise("off-peak"))

# Afficher le DataFrame avec les heures de pointe identifiées :
display(spark_df)

In [0]:
from pyspark.sql.functions import count

result_df = spark_df.groupBy("Route").agg(
    avg("Delay").alias("RetardMoyen"),
    avg("Passengers").alias("NombrePassagersMoyen"),
    count("*").alias("NombreTotalVoyages")
)

#Afficher le DataFrame résultant :
display(result_df)